# Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk import word_tokenize, pos_tag

from string import punctuation
import re


import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

import spacy

!pip install stop-words
from stop_words import get_stop_words

from textblob import TextBlob

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 12.0 MB 504 kB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=064280e41c5f5948080db437b997e72b721c0577e6ef6ecaff6c26cb5659892b
  Stored in dire

# Парсим ссылку, получаем заголовок, текст

In [ ]:
# Парсинг

# Генерируем признаки

In [3]:
# загружаем модель
from google.colab import files
uploaded = files.upload()

Saving итоговая модель.h5 to итоговая модель.h5
Saving модель_для_анализа_заголовка.h5 to модель_для_анализа_заголовка.h5


In [43]:
# анализ заголовка
def headlineAnalis(headline):
    model_for_headline = keras.models.load_model('модель_для_анализа_заголовка.h5')

    df_h = pd.DataFrame({'Headline' : [headline]})

    oh_headlines = [one_hot(words, 5000) for words in df_h['Headline']]
    x = pad_sequences(oh_headlines, padding='pre', maxlen=30)

    headLines = model_for_headline.predict(x)
    df_h.Headline = headLines

    return df_h

# Работа с текстом

In [5]:
# удаление пунктуации и привеение к нижнему регистру

def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ',
             43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ',
             63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def cleanText(text):
    text = text.lower()
    
    text = remove_punct(text)

    signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', '@', '>', '<', '–', '&', '%', '•', '/', '“']
    for punct_sign in signs:
        text = text.replace(punct_sign, ' ')

    text = text.replace('\n', '')
    text = text.replace('\r', '')
    text = re.sub(r"\d+", "", text, flags=re.UNICODE)
    text = text.replace("  " , " ")
    text = text.replace("  " , " ")

    return text

In [6]:
# подсчет количества наречий

def amountAdverbs(text):
    amountADV = 0
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    for i in tags:
        if len(i) == 2:
            if i[1] == 'RB':
                amountADV += 1
        else:
            continue
    return amountADV

In [20]:
# лемматизация и удаление стоп-слов

def removeStopWords_and_lemmitizeText(text):
    lemmatizer = spacy.load('en', disable=['parser', 'ner'])
    doc = lemmatizer(text)
    new_text = " ".join([token.lemma_ for token in doc\
                         if token.lemma_ not in get_stop_words('english')\
                         and (token.lemma_ != "-PRON-")\
                         and (len(token.lemma_) != 1 or token.lemma_ == 'i')])
    # new_text = new_text.replace("-PRON-", '')
    # new_text = new_text.replace("s", '')
    return new_text

In [32]:
# тональность и субъективность

def polarityAndSubjectivityOfText(text):
    pol = TextBlob(text).polarity
    sub = TextBlob(text).subjectivity
    return pol, sub

In [39]:
# векторизация

def toVecText(text):

    d = {'text': [text]}
    df1 = pd.DataFrame(d)

    oh_text = [one_hot(words, 5000) for words in df1.text]
    embedded_doc = pad_sequences(oh_text, padding='pre', maxlen=800)
    df_text = pd.DataFrame(embedded_doc)

    return df_text

# oh_text = one_hot(text, 5000)
# embedded_doc = pad_sequences(oh_text, padding='pre', maxlen=800)
# df_text = pd.DataFrame(embedded_doc)
# df_new = pd.concat([df.drop(['Body_02'], axis = 1), df_text], axis = 1)

In [40]:
# from numpy.ma.core import logical_not
def allTextPreparation(text):

    # чистим от пунктуации + нижгий регистр
    text = cleanText(text)

    # считаем количество наречий
    amountAdv = amountAdverbs(text)

    # лемматизация и удаление стоп-слов
    text = removeStopWords_and_lemmitizeText(text)

    # полярность и субъективность
    l_pol_sub = polarityAndSubjectivityOfText(text)

    polarity = l_pol_sub[0]
    subjectivity = l_pol_sub[1]

    # векторизация текста
    vec_text = toVecText(text)

    data_ = {'Amount Adverbs' : [amountAdv], 
             'Polarity' : [polarity],
             'Subjectivity' : [subjectivity]}

    df_features = pd.DataFrame(data_)

    df_new = pd.concat([df_features, vec_text], axis = 1)

    return df_new


# Итоговая функция для подготовки данных

In [10]:
# uploaded = files.upload()

Saving new_big_DATA-000.csv to new_big_DATA-000.csv


In [12]:
# df_1 = pd.read_csv('new_big_DATA-000.csv')
# df_1

,title,publication,date,url,content
0,Fire tears through Dubai luxury hotel near pla...,Fox News,2015-12-31,https://web.archive.org/web/20160101000951/htt...,More than a dozen people were hurt after mass...
1,Top Carson campaign staffers resign,Fox News,2015-12-31,https://web.archive.org/web/20160101000951/htt...,Republican presidential candidate Ben Carson’...
2,Democrat Martin O’Malley fails to qualify for ...,Fox News,2015-12-31,https://web.archive.org/web/20160101000951/htt...,Democratic presidential candidate Martin O’Ma...
3,"Brawl breaks out in GOP race, below the Trump ...",Fox News,2015-12-30,https://web.archive.org/web/20160101000951/htt...,A nasty battle has broken out in the Republic...
4,Donald Trump blasts Bill Clinton as ’one of th...,Fox News,2015-12-30,https://web.archive.org/web/20160101000951/htt...,Donald Trump launched new attacks against Bil...
...,...,...,...,...,...
4995,"Act of Love, Part II",National Review,2017-02-17,http://www.nationalreview.com/article/444992/d...,Editor’s Note: In the February 20 issue of Nat...
4996,Remembering Michael Novak,National Review,2017-02-17,http://www.nationalreview.com/article/445026/m...,Michael Novak loved the Catholic Church and th...
4997,Michael Novak: Intellectual Godfather to a Gen...,National Review,2017-02-17,http://www.nationalreview.com/article/445031/m...,Next time your work is frustrating or unpleasa...
4998,Air It Out,National Review,2017-02-17,http://www.nationalreview.com/article/445010/m...,In an environment where every day has felt lik...


In [47]:
def dataPreparation(data):

    # data = [text, headline]
    text = data[0]
    headline = data[1]
    df_text = allTextPreparation(text)
    df_headline = headlineAnalis(headline)

    df = pd.concat([df_headline, df_text], axis = 1)

    return df


In [48]:
# name = df_1['title'][0]
# text_ = df_1['content'][0]
# data = [text_, name]
# df_f = dataPreparation(data)
# df_f

,Headline,Amount Adverbs,Polarity,Subjectivity,0,1,2,3,4,5,...,790,791,792,793,794,795,796,797,798,799
0,1.0,13,0.198182,0.500606,0,0,0,0,0,0,...,3640,4180,241,4651,3946,4131,3526,2233,4061,239


# Предсказывание результата

In [49]:
def dataPrediction(data_df):
    finalModel = keras.models.load_model('итоговая модель.h5')
    result = finalModel.predict(data_df)
    result = int(np.around(result))

    if result == 0:
        return "С высокой долей вероятности новость - фейк."
    elif result == 1:
        return "С высокой долей вероятности новость не является фейком."
    

# Сборка

In [ ]:
def fakeNewsChecking(url):
    # отправляем url в парсер, получаем данные
    # data = parser(url)
    df = dataPreparation(data)
    res = dataPrediction(df)
    # print (res)
